In [1]:
from bs4 import BeautifulSoup
import math,pickle,textwrap,random,os,lxml,plotly
import pandas as pd
from datetime import date,datetime
import plotly.graph_objects as go
import plotly.express as px
from pprint import pprint
import numpy as np
from helper import *

# Load data

In [2]:
with open('../data/location_mapping.pkl', 'rb') as f:
    mapping_colation = pickle.load(f)

In [3]:
with open('../data/raw/1791_Purm_Louw_04.xml','r') as f:
    file = f.read()
purmLouw_1791 = BeautifulSoup(file,'xml')

In [4]:
purmLouw_1791_meta = pd.read_excel('../data/raw/1791_Purm_Louw_04.xlsx',sheet_name='datum',engine='openpyxl')
purmLouw_1791_meta_all = pd.read_excel('../data/raw/1791_Purm_Louw_04.xlsx',sheet_name='Overview',engine='openpyxl')
purmLouw_1791_meta.head()

,Value,Context,Imagename,Doc,Page,Region,Line,Word,datum,offset,length,continued
0,1775,1775 tot 1788 enz:;,NL-HlmNHA_143_21_0003.jpg,901446,3,r1,r1l10,NaN,1775-xx-xx,0,4,False
1,1788,1775 tot 1788 enz:;,NL-HlmNHA_143_21_0003.jpg,901446,3,r1,r1l10,NaN,1788-xx-xx,8,6,False
2,1785,"der Stad Purmerende, 1785",NL-HlmNHA_143_21_0012.jpg,901446,12,region_1568800990264_4443,line_1568800996505_4451,NaN,1785-xx-xx,21,4,False
3,1785,"der Stad Purmerende, 1785",NL-HlmNHA_143_21_0013.jpg,901446,13,r2,r2l3,NaN,1785-xx-xx,21,4,False
4,1785,merende in 't Jaar 1785.,NL-HlmNHA_143_21_0014.jpg,901446,14,r1,r1l14,NaN,1785-xx-xx,19,4,False


# process data

## process excel & xml
* get year-month-day in metadata excel
* generate a dictionary, keys are each ab in chronicle, value is a dictionary, contains:                 
  'type', 'lb', 'line', 'value', 'belong_to_date', 'page', 'whole_line_context'

In [5]:
# preprocess excel metadata of each chronicle
for row in purmLouw_1791_meta.index.tolist():
     if type(purmLouw_1791_meta.loc[row,'datum']) is not str:
        try:
            purmLouw_1791_meta.loc[row,'datum'] = purmLouw_1791_meta.loc[row,'when']
        except:
            pass
            # print(purmLouw_1791_meta.loc[row,'datum'])


In [6]:
# preprocess excel metadata of each chronicle
_ = purmLouw_1791_meta.loc[purmLouw_1791_meta['datum'].notnull()]
for row in _.index.tolist():
    date = purmLouw_1791_meta.loc[row,'datum']
    try:
        if (type(date) == str) & (len(date.split('-')) == 3):
            purmLouw_1791_meta.loc[row,'year'],purmLouw_1791_meta.loc[row,'month'],purmLouw_1791_meta.loc[row,'day'] = date.split('-')
    except:
        print(date)

In [7]:
# for each chronicle excel file:
# Completeness of dates: year, month ,day
month_miss = purmLouw_1791_meta.loc[purmLouw_1791_meta['month']=='xx',:].shape[0]
day_miss = purmLouw_1791_meta.loc[purmLouw_1791_meta['day']=='xx',:].shape[0]
print(f"missing month: {month_miss}, missing days: {day_miss}")

# fill missing to 01
purmLouw_1791_meta['month'] = purmLouw_1791_meta['month'].str.replace('xx','01') # .astype('Int64')
purmLouw_1791_meta['day'] = purmLouw_1791_meta['day'].str.replace('xx','01') #.astype('Int64')

missing month: 38, missing days: 67


In [8]:
purmLouw_1791_meta.shape

(487, 15)

In [9]:
# process xml data of each chronicle
# generate a dictionary 
purm_louw_dic = {}
locatie_count_ab = 0
abs = purmLouw_1791.find_all('ab')# limit=100  # {'facs':['#facs_1_region_1','#facs_2_region_1']})
for ab in abs:
    for ind,i in enumerate(ab):
        if i.name in ['locatie','datum']:
            lb_fac = i.findPrevious('lb')['facs'] 
            ind = str(ind) + '|' + lb_fac 
            purm_louw_dic[ind] = {'type':i.name}
            purm_louw_dic[ind].update({'lb':lb_fac})
            purm_louw_dic[ind].update({'line':i.findPrevious('lb')['n']})
            purm_louw_dic[ind].update({'value':i.text})
            # find the whole line in metadata excel
        if i.name == 'locatie':
            locatie_count_ab += 1
            try:
                # print(i.find_previous('datum').findPrevious('lb'))
                purm_louw_dic[ind].update({'belong_to_date':{'text':i.find_previous('datum').text, 
                                                             'date_line':i.find_previous('datum').findPrevious('lb')['facs']}})
            except:
                print(i)

 

<locatie>P U R M E R E N D E</locatie>


In [10]:
print(len(purm_louw_dic))
print(locatie_count_ab)

1419
724


In [63]:
# add page and line
# normalized date
# context
cannot_get_date_inmeta = 0
cannot_get_page = 0
locatie_count = 0
no_belong_to_date = 0
for k,v in purm_louw_dic.items():
    page,line = get_page_line(v['lb'])
    try:
        purm_louw_dic[k].update({'page':page,'line':line,
                                'whole_line_context':purmLouw_1791_meta_all.query("Page ==@ page").query("Line==@line")['Context'].tolist()[0]})
    except:
        cannot_get_page += 1
        # print(page, line)
        # metadata not compelet (excel)
    
    if v['type'] == 'locatie':
        locatie_count += 1
        try:
            date_value = str(v['belong_to_date']['text'])# .strip()
        except:
            no_belong_to_date += 1
        try:
            date_page, date_line = get_page_line(v['belong_to_date']['date_line'])
            da = purmLouw_1791_meta.query("Page ==@ date_page").query("Line==@ date_line").query("Value==@date_value")
            purm_louw_dic[k]['belong_to_date'].update({'normal_date':f"{da['year'].values[0]}-{da['month'].values[0]}-{da['day'].values[0]}"})            
        except:
            cannot_get_date_inmeta +=1
            



In [64]:
print(cannot_get_page)
print(cannot_get_date_inmeta)
print(locatie_count)
print(no_belong_to_date)

248
356
724
1


In [65]:
pprint(purm_louw_dic['21|#facs_21_r1l10'])

{'belong_to_date': {'date_line': '#facs_20_r2l3', 'text': 'tus 1786'},
 'lb': '#facs_21_r1l10',
 'line': 'r1l10',
 'page': 21,
 'type': 'locatie',
 'value': 'Purmerend',
 'whole_line_context': 'Dan zal Ons Purmerend,'}


## df for plot

In [66]:
purm_louw_dic['38|#facs_53_r1l14']
mapping_colation.get(purm_louw_dic['38|#facs_53_r1l14']['value'].strip(' ').lower())

{'match_place': 'Utrecht', 'la': 52.100140025887015, 'lo': 5.084805569558331}

In [67]:
purm_louw_dic['38|#facs_53_r1l14']['value'].strip(' ').lower()

'utregt'

In [68]:
df = pd.DataFrame(columns=['la','lo','hover','date_belong'])
count_all_locatie = 0
count_hover_locatie = 0
for k,v in purm_louw_dic.items():
    if v['type'] == 'locatie':
        count_all_locatie += 1
        try: 
            place_pack = mapping_colation.get(v['value'].strip(' ').lower())
            if place_pack['la'] in df['la'].tolist():
                df.loc[k,'la'] = place_pack['la'] + random.uniform(0,0.01)
            else:
                df.loc[k,'la'] = place_pack['la']
            if place_pack['lo'] in df['lo'].tolist():
                df.loc[k,'lo'] = place_pack['lo'] + random.uniform(0,0.01)
            else:
                df.loc[k,'lo'] = place_pack['lo']
            # hover info
            lb = v['lb']
            tt = f"<b>location: {place_pack['match_place']}</b><br><br><b>page and line: {lb}<br><br>{v['whole_line_context']}" 
            df.loc[k,'hover'] = tt
            df.loc[k,'date_belong'] = v['belong_to_date']['normal_date']
            df.loc[k,'location'] = place_pack['match_place']
            df.loc[k,'raw_location'] = v['value']
            count_hover_locatie +=1
        except:
            # print(v['value'],lb,v['whole_line_context'])
            pass
        

In [69]:
print(count_all_locatie,count_hover_locatie)

724 301


In [70]:
# df.loc[df['date_belong'].notnull()].shape # notnulldate: 301 / 590
df.head() # 590


,la,lo,hover,date_belong,location,raw_location
10|#facs_3_r1l5,NaN,NaN,<b>location: nan</b><br><br><b>page and line: ...,NaN,NaN,NaN
9|#facs_5_r4l4,52.509903,4.946108,<b>location: Purmerende</b><br><br><b>page and...,1788-01-01,Purmerende,Purmerende
53|#facs_5_r4l23,54.063861,19.659497,<b>location: Hollant</b><br><br><b>page and li...,1788-01-01,Hollant,Holland
46|#facs_6_r3l31,54.066678,19.664917,<b>location: Hollant</b><br><br><b>page and li...,1788-01-01,Hollant,Holland
40|#facs_7_r3l23,54.072164,19.661433,<b>location: Hollant</b><br><br><b>page and li...,1788-01-01,Hollant,Holland


In [76]:
purm_louw_dic['14|#facs_35_r1l2'] # #facs_35_r1l2

{'type': 'locatie',
 'lb': '#facs_35_r1l2',
 'line': 'r1l2',
 'value': 'de doelen',
 'belong_to_date': {'text': '12 Octob:',
  'date_line': '#facs_35_line_1568138097729_1400'},
 'page': 35,
 'whole_line_context': 'stadig de Wagt, voor en bij de doelen; en'}

In [77]:
df.loc[df['date_belong'].isnull()].shape
# df.loc[df['raw_location'].isna(),:]
# df.loc['38|#facs_53_r1l14','hover']

(289, 6)

In [92]:
df = df.loc[df['la'].notna()&(df['lo'].notna())&((df['date_belong'].notna())),:]
df.shape

(281, 6)

In [99]:
df['date_belong'][0]

'1788-01-01'

In [94]:
pack(object=df,
     file_name='test_df.pkl',mode='wb')
# df.loc[df['date_belong'].notnull()].sort_values(by='date_belong')

In [110]:
df = pack(file_name='test_df.pkl',mode='rb')
df = df.loc[df['date_belong'].notnull()].sort_values(by='date_belong')
df.shape

(281, 6)

In [115]:
for i in df['date_belong'].tolist():
    try:
        [int(j) for j in i.split('-')]
    except:
        print(i)


nan-nan-nan
nan-nan-nan


In [126]:
df_ = df.loc[df['date_belong']!= 'nan-nan-nan',:]

for i in df_['date_belong'].tolist():
    try:
        [int(j) for j in i.split('-')]
    except:
        print(i)

In [127]:
pack(object=df_, file_name='test_df.pkl',mode='wb')

In [128]:
df_.loc[df_['date_belong'] == 'nan-nan-nan',:]

,la,lo,hover,date_belong,location,raw_location


In [134]:
def string_to_date(string):
    # 1778-10-13
    _ = string.split('-')
    assert len(_) == 3, f"{string} is not standard format for datetime.date()"
    try: 
        d = datetime.date(int(_[0]),int(_[1]),int(_[2]))
        return d
    except:
        print(f"{_} ???")
date_list = []
for i in df_['date_belong'].tolist():
    date_list.append(string_to_date(i))# date(*[int(j) for j in i.split('-')]))
df_['date_belong'] = date_list
date_list = sorted(date_list)


<ipython-input-134-87246180de6a>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_['date_belong'] = date_list


In [136]:
print(date_list[10],date_list[4])

1778-07-01 1766-01-01


In [138]:
for i in date_list:
    if (i <= date_list[10]) & (i >= date_list[4]):
        print(i)

1766-01-01
1766-01-01
1766-01-01
1766-01-01
1773-07-20
1778-01-01
1778-01-01
1778-07-01
1778-07-01


In [ ]:
# uni_dates = set([v.get('datetime') for k,v in facs2re1_dic.items() if v.get('datetime')])

color = plotly.colors.qualitative.Plotly
Fig = go.Figure()
for ind,i in enumerate(sorted(df['date_belong'].unique())):
    _ = df.loc[df['date_belong']==i,:]
    d = go.Scattermapbox(lat = _['lat'],
                         lon = _['lon'],
                         mode='markers',
                         marker = dict(size=12,color=color[ind]), # go.scattermapbox.Marker
                         text = _['hover'].tolist(),
                         name = str(i),
                         hoverinfo='text')
    Fig.add_trace(d)

Fig.update_layout(mapbox=dict(style='open-street-map'))
Fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
Fig.update_layout(legend=dict(y=0.2,x=1))
Fig.show()
# Fig.write_html('test.html')